In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd

In [12]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
website = 'https://desaparecidos.pjc.mt.gov.br/inicio'
driver.get(website)

driver.maximize_window()

In [13]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [14]:
nomes = []
idades = []
local_desaparecimento = []


try:
    while True:
        # Espera até que o conteúdo dinâmico da página seja carregado
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "individuo-card.ng-star-inserted"))
        )
        
        # Agora que o conteúdo está carregado, crie uma sopa com o Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        for individuo_card in soup.find_all('individuo-card', class_='ng-star-inserted'):
            # Obtém o nome de <mat-card-title>
            mat_card_title = individuo_card.find('mat-card-title', class_='mat-card-title')
            if mat_card_title:
                nomes.append(mat_card_title.get_text(strip=True))
            
            # Obtém a idade do <span> imediatamente após <mat-card-subtitle>
            mat_card_subtitle = individuo_card.find('mat-card-subtitle', class_='mat-card-subtitle')
            if mat_card_subtitle and mat_card_subtitle.next_sibling:
                idades.append(mat_card_subtitle.next_sibling.get_text(strip=True))
            
            # Obtém o local de desaparecimento do <span> com class='ng-star-inserted'
            span_location = individuo_card.find('span', class_='ng-star-inserted')
            if span_location:
                local_desaparecimento.append(span_location.get_text(strip=True))

        # Tenta encontrar o botão 'next'
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "li.pagination-next.ng-star-inserted a")
            if next_button.is_enabled():
                next_button.click()
                time.sleep(2)  # Espera um tempo para a nova página carregar
            else:
                break  # Se o botão 'next' não estiver habilitado, saia do loop
        except NoSuchElementException:
            break  # Se não encontrar o botão 'next', saia do loop

except TimeoutException:
    print("O tempo de carregamento da página excedeu o limite estabelecido.")


# Imprime as listas finais
print("Nomes:", nomes)
print("Idades:", idades)
print("Local de desaparecimento:", local_desaparecimento)

Nomes: ['RONALTH DE SOUSA OLIVEIRA', 'VALDERSON WILSON GUIMARAES', 'MARIA VITORIA BARRETO', 'ARTHUR MORAES ALVES', 'RODRIGO TRAPIÁ DE OLIVIERA', 'ROBSON FELIPE LIMA DOS SANTOS', 'ADELINO RODRIGUES DE SOUZA', 'ALINE FRANCISCO DE SOUZA', 'THAMILA RODRIGUES DE ARAÚJO', 'ANA BEATRIZ SANTOS', 'ROBERT NEIVERT SOUZA SANTOS', 'EDSON DOS SANTOS SANTANA', 'AKCEL LOPES CAMPOS', 'MARIANA DE LIMA ALVES FERREIRA', 'MARTINS MARQUES DUARTE', 'BEATRIZ VITORIA DE OLIVEIRA CASTRO', 'BRENO JUNIOR DE OLIVEIRA CASTRO', 'JEFFERSON CAMPOS DE OLIVEIRA', 'LEONARDO PIRES RANGEL', 'ADENILDA BENEDITA LOPES', 'JOENIL JUNIOR SILVA ROSA', 'BRUNO HENRIQUE LIMA MONTE FERRAZ', 'JOÃO RAMON MESSA RATIER', 'SABRINA GOMES DA SILVA', 'WILLIAM GUILHERME WADA', 'EDIVAM BARBOSA DA SILVA', 'MICHELY CAVANHA SIQUEIRA', 'WALISSON WILKER RODRIGUES BORGES', 'CARLOS MATHEUS RIBEIRO DE CAMPOS', 'THYAGO DA SILVA KLEY', 'RODRIGO AMARAL DA SILVA', 'ERON MIGUEL BEZERRA SILVA JUNIOR', 'ALESSANDRO DA SILVA', 'VALDETE DA SILVA VALE', 'VALDINE

In [19]:
data = {
    'Nomes': nomes,
    'Idades': idades,
    'Local de Desaparecimento': local_desaparecimento
}

df = pd.DataFrame(data)
df.to_csv('Desaparecidos Polícia Civil MT.csv', index=False)